In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("orders.csv")
df

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,18/12/21 10:23,SELL,SBI,CNC,1000/1000,525.25,COMPLETE
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000/1000,125.70,COMPLETE
2,16/12/21 15:08,BUY,SBI,CNC,1000/1000,520.80,COMPLETE
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250/250,490.55,COMPLETE
4,16/12/21 13:54,BUY,ASHOKLEY,MIS,0/1000,127.10,CANCELLED
5,16/12/21 13:21,SELL,TATAMOTORS,MIS,250/250,492.10,COMPLETE
6,16/12/21 12:51,BUY,TATAMOTORS,MIS,0/250,490.80,CANCELLED
7,16/12/21 12:39,SELL,ASHOKLEY,MIS,1000/1000,125.96,COMPLETE
8,16/12/21 12:29,BUY,ASHOKLEY,MIS,2000/2000,125.70,COMPLETE
9,16/12/21 11:22,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE


In [19]:
#1 Different types of charges for Individual trade

df1=df.copy()
cncdrop = df1[(df1["Product"]=="CNC")].index
df1.drop(cncdrop,inplace=True)
df1.drop(["Time"], axis=1,inplace=True)
df1["Qty."]=df1["Qty."].apply(lambda x:x.split("/")[0]).astype(int, False)
df1["Turnover"]=df1["Qty."]*df1["Avg. price"]
df1["Brokerage"]= df1["Turnover"].apply(lambda x: (x*0.0003) if x*0.0003<20 else 20)
df1['STT/CTT'] = np.where(df1['Type'] == 'SELL', df1['Turnover'] * 0.00025,0)
df1["ETC"]= df1["Turnover"]*0.0000325
df1["SEBI"]= df1["Turnover"]*0.000001
df1["GST"]= (df1["Brokerage"]+df1["ETC"]+df1["SEBI"])*0.18
df1['Stamp charges'] = np.where(df1['Type'] == 'BUY', df1['Turnover'] * 0.00003,0)
df1["Total charges"] = df1["Brokerage"]+df1["ETC"]+df1["SEBI"]+df1["GST"]+df1["STT/CTT"]+df1["Stamp charges"]
df1['Gross PNL']= np.where(df1['Type'] == 'SELL', df1['Turnover']*(1), df1['Turnover']*(-1))
df1['Net PNL']= df1['Gross PNL']-df1['Total charges']
df01= df1.copy()
df01.drop(['Gross PNL'], axis=1, inplace=True)
df01.drop(['Net PNL'], axis=1, inplace=True)
df01

,Type,Instrument,Product,Qty.,Avg. price,Status,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp charges,Total charges
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE,125700.0,20.0,0.00000,4.085250,0.125700,4.357971,3.771000,32.339921
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE,122637.5,20.0,0.00000,3.985719,0.122637,4.339504,3.679125,32.126985
4,BUY,ASHOKLEY,MIS,0,127.10,CANCELLED,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE,123025.0,20.0,30.75625,3.998312,0.123025,4.341841,0.000000,59.219428
6,BUY,TATAMOTORS,MIS,0,490.80,CANCELLED,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE,125960.0,20.0,31.49000,4.093700,0.125960,4.359539,0.000000,60.069199
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE,251400.0,20.0,0.00000,8.170500,0.251400,5.115942,7.542000,41.079842
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.97500,8.186750,0.251900,5.118957,0.000000,96.532607
10,SELL,ASHOKLEY,MIS,0,124.45,CANCELLED,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.97500,8.186750,0.251900,5.118957,0.000000,96.532607


In [18]:
#Stock wise and Type wise analysis with weighted Avg. price & calculated charges

df2=df1.copy()
df2=df2.groupby(['Instrument','Type']).sum() 
df2['Weighted_Avg_Price'] = df2["Turnover"] / df2["Qty."]
df2

Qty.  Avg. price  Turnover  Brokerage    STT/CTT        ETC  \
Instrument Type                                                                
ASHOKLEY   BUY   5000      629.75  628350.0       80.0    0.00000  20.421375   
           SELL  5000      502.31  629760.0       60.0  157.44000  20.467200   
TATAMOTORS BUY    250      981.35  122637.5       20.0    0.00000   3.985719   
           SELL   250      492.10  123025.0       20.0   30.75625   3.998312   

                     SEBI        GST  Stamp charges  Total charges  Gross PNL  \
Instrument Type                                                                 
ASHOKLEY   BUY   0.628350  18.188950      18.850500     138.089176  -628350.0   
           SELL  0.629760  14.597453       0.000000     253.134413   629760.0   
TATAMOTORS BUY   0.122637   4.339504       3.679125      32.126985  -122637.5   
           SELL  0.123025   4.341841       0.000000      59.219428   123025.0   

                       Net PNL  Weighted_Avg_Price  
Instrument Type                                     
ASHOKLEY   BUY  -628488.089175             125.670  
           SELL  629506.865587             125.952  
TATAMOTORS BUY  -122669.626985             490.550  
           SELL  122965.780572             492.100

In [17]:
#Overall Summary of each Stocks

df3=df1.copy()
df3=df3.groupby(['Instrument']).sum() 
df3['% Charges on Gross PNL'] = (df3['Total charges'] / df3['Gross PNL']) * 100
df3=df3[['Gross PNL', 'Total charges', 'Net PNL', '% Charges on Gross PNL']]
df3

,Gross PNL,Total charges,Net PNL,% Charges on Gross PNL
Instrument,,,,
ASHOKLEY,1410.0,391.223588,1018.776412,27.746354
TATAMOTORS,387.5,91.346414,296.153586,23.573268


In [ ]:
with pd.ExcelWriter("stockassingment(3).xlsx")as writer:
    df01.to_excel(writer, sheet_name='output1',index= False)
    df2.to_excel(writer, sheet_name='output2')
    df3.to_excel(writer, sheet_name='output3')